# **Implementação da Resolução do Problema do Turismo pelo Brasil**

# Lendo os dados

Leremos as tabelas csv utilizando uma lista de listas interpretando-a como uma matriz.

In [ ]:
from google.colab import files # carregar os arquivos 'conexCapitais.csv' e 'distCapitais.csv'
uploaded = files.upload()

Saving conexCapitais.csv to conexCapitais.csv
Saving distCapitais.csv to distCapitais.csv


In [ ]:
!ls   # checar se o arquivo foi devidamente carregado

conexCapitais.csv  distCapitais.csv  sample_data


In [ ]:
import csv
reader = csv.reader(open('conexCapitais.csv', newline=''))
conexMat = list(reader)
reader = csv.reader(open('distCapitais.csv', newline=''))
distMat = list(reader)

# Convertemos as strings para int
for i in range(1, 28):
  for j in range(1, 28):
    if distMat[i][j] == '\xa0':
      distMat[i][j] = -1
    else:
      distMat[i][j] = int(distMat[i][j])

Renomeamos algumas legendas para normalizar os nomes das cidades.

In [ ]:
conexMat[3][0] = 'Belo Horizonte'
distMat[3][0] = 'Belo Horizonte'
conexMat[6][0] = 'Campo Grande'
distMat[6][0] = 'Campo Grande'
conexMat[22][0] = 'Rio de Janeiro'
distMat[22][0] = 'Rio de Janeiro'
conexMat[24][0] = 'São Luís'
distMat[24][0] = 'São Luís'
conexMat[0][22] = 'Rio de Janeiro'
distMat[0][22] = 'Rio de Janeiro'
conexMat[0][24] = 'São Luís'
distMat[0][24] = 'São Luís'

# Estruturando os dados e criando funções utilitárias


Vamos construir agora um dicionário relacionando uma cidade (usando seu nome como chave) a um conjunto com todas as outras conectadas diretamente a ela. Não faz sentido dizer que uma cidade está conectada a si mesma, então vamos desconsiderar esse caso.


In [ ]:
direcConnected = dict()
for i in range(1, len(conexMat)):
  direcConnected.update({conexMat[i][0]: set()}) ## {nomedacidade : conjuntovazio}
  for j in range(1, len(conexMat)):
    if conexMat[i][j] == '1' and i != j: # se a cidade de nome conexMat[i][0] está conectada à cidade de nome conexMat[0][j]
      direcConnected[conexMat[i][0]].add(conexMat[0][j])

In [ ]:
direcConnected

{'Aracaju': {'Maceió', 'Salvador'},
 'Belo Horizonte': {'Brasília', 'Rio de Janeiro', 'São Paulo'},
 'Belém': {'Macapá', 'Manaus', 'São Luís'},
 'Boa Vista': {'Manaus'},
 'Brasília': {'Belo Horizonte',
  'Goiânia',
  'Manaus',
  'Palmas',
  'Salvador',
  'São Paulo'},
 'Campo Grande': {'Cuiabá', 'Goiânia', 'Porto Alegre', 'São Paulo'},
 'Cuiabá': {'Campo Grande', 'Goiânia', 'Porto Velho'},
 'Curitiba': {'Florianópolis', 'Porto Alegre', 'São Paulo'},
 'Florianópolis': {'Curitiba', 'Porto Alegre'},
 'Fortaleza': {'Goiânia', 'Natal', 'Salvador', 'Teresina'},
 'Goiânia': {'Brasília', 'Campo Grande', 'Cuiabá', 'São Paulo'},
 'João Pessoa': {'Natal', 'Recife'},
 'Macapá': {'Belém'},
 'Maceió': {'Aracaju', 'Recife'},
 'Manaus': {'Belém', 'Boa Vista', 'Brasília', 'Porto Velho', 'Rio Branco'},
 'Natal': {'Fortaleza', 'João Pessoa'},
 'Palmas': {'Belém', 'Brasília'},
 'Porto Alegre': {'Campo Grande', 'Curitiba', 'Florianópolis'},
 'Porto Velho': {'Cuiabá', 'Manaus'},
 'Recife': {'João Pessoa', '

Podemos obter o conjunto de cidades conectadas a Maceió com:

```
direcConnected['Maceió']
```


Criaremos funções auxiliares: **stepCost('Estado inicial', 'Ação')** retorna o custo da ação de transição entre dois estados, **h('Estado')** retorna o valor da heurística para um dado estado e **objectiveTest('Estado')** retorna Verdadeiro ou Falso para se o objetivo foi atingido.

In [ ]:
def stepCost(state, action):
  global direcConnected
  listOfStates = list(direcConnected)
  
  # Finding the position of the states in distMat
  for i in range(0, len(listOfStates)):
      if state == listOfStates[i]:
        break
  for j in range(0, len(listOfStates)):
    if action == listOfStates[j]:
      break   

  if (i >= j):
    return distMat[i+1][j+1]
  else:
    return distMat[j+1][i+1]

def h(state):
  global objective
  global direcConnected
  listOfStates = list(direcConnected)
  
  # Finding the position of the states in distMat
  for i in range(0, len(listOfStates)):
      if state == listOfStates[i]:
        break
  for j in range(0, len(listOfStates)):
    if objective == listOfStates[j]:
      break   
 
 # Remember that distMat has a line and a column of captions
  if (i < j):
    return distMat[i+1][j+1]
  else:
    return distMat[j+1][i+1]

def objectiveTest(state):
  global objective
  return state == objective

A partir do momento em que o estado objetivo está definido, podemos usar as funções definidas acima com:

```
h('Goiânia')
objectiveTest('Rio de Janeiro')
```

# Definindo as buscas cegas

**Busca de custo uniforme**

In [ ]:
import heapq
import time

# Searchs nodes by minimum  path cost
# Returns None if there is no solution
# Returns (path cost, [initial state, ...,  objective]) if finds the solution
def uniformCostSearch(state):
    global nodesUniformCost # to count the amount of nodes generated
    global direcConnected # to access all the possible actions of a state

    node = {'state' : state, 'path_cost' : 0, 'came_from' : state}
    frontier = [] # we use a heap queue ordered by the path cost
    explored = {} # dictionary with {name_of_state : node} pairs
    
    # time.perf_counter() is in the tuple to solve drawed comparisons between paths of equal costs
    heapq.heappush(frontier, (node['path_cost'], time.perf_counter(), node))
    nodesUniformCost += 1
    
    while True:
      # If there are no more nodes to explore and we didn't find the solution
      if (len(frontier) == 0): 
        return None
      
      # Picking the node with smaller path cost
      node = (heapq.heappop(frontier))[2] 
      explored.update({node['state'] : node})
      nodesUniformCost += 1

      # If we achieved the objective, return the path and total path cost
      if (objectiveTest(node['state'])):
        solution = []
        solution_cost = node['path_cost']

        while node['path_cost'] != 0: # path_cost = 0 only for stater node
          solution.insert(0,node['state'])
          node = explored[node['came_from']]
        solution.insert(0,node['state'])

        return (solution_cost, solution)

      # Getting all actions the state can do
      actions = direcConnected[node['state']]
      for son_state in actions:
        son_node = {'state': son_state, 'path_cost' : node['path_cost'] + stepCost(node['state'], son_state), 'came_from' : node['state']}

        # Search if son is in explored
        in_explored = False
        if son_state in explored:
          in_explored = True

        # Search if son is in frontier
        in_frontier = False
        for frontier_node in frontier:
          if frontier_node[2]['state'] == son_state:
            in_frontier = True
            break
        
        # If son isn't already explored nand on frontier we add it to the frontier
        if not in_frontier and not in_explored:
          heapq.heappush(frontier, (son_node['path_cost'], time.perf_counter(), son_node))

        # If son is already explored, but this new path is smaller
        elif in_frontier and frontier_node[0] > son_node['path_cost']:
          frontier_node = (son_node['path_cost'], time.perf_counter(), son_node)
          heapq.heapify(frontier)

**Busca de aprofundamento iterativo**

In [ ]:

# Searchs nodes by maximum deepning within a depth_limit
# Returns (None) if there is no answer in any limit
# Returns (maximum_depth, None) with the maximum_depth achieved by a frontier node if there is no answer within that limit
# Returns (path cost, [initial state, ...,  objective]) if finds the solution
def limitedDeepeningSearch(state, depth_limit):
  global nodesIterativeDeepening # to count the amount of nodes generated
  global direcConnected # to access all the possible actions of a state
  
  node = {'state' : state, 'path_cost' : 0, 'came_from' : state}
  node_depth = maximum_depth = 0
  frontier = [] # we use a stack with the newer nodes in front
  explored = {} # dictionary with {name_of_state : node} pairs
  frontier.append((node_depth, node))

  while True:
    # If there are no more nodes to explore and we didn't find the solution
    if (len(frontier) == 0):
      # If we couldn't explore every path possible
      if (maximum_depth > depth_limit):
        return (maximum_depth, None)
      # If we did explore every path possible
      else:
        return None
    
    # Picking the deepest node
    node_depth, node = frontier.pop()
    nodesIterativeDeepening += 1
    
    # Updating the maximum depth achieved by a frontier node
    if node_depth > maximum_depth:
      maximum_depth = node_depth

    # Ignores nodes that surpasses the limit
    if node_depth > depth_limit:
      continue
    
    # We can only say that a node was explored if it was within the depth limit
    explored.update({node['state'] : node})

    # If we achieved the objective, return the path and total path cost
    if objectiveTest(node['state']):
      solution = []
      solution_cost = node['path_cost']

      while node['path_cost'] != 0: # path_cost = 0 only for stater node
        solution.insert(0,node['state'])
        node = explored[node['came_from']]
      solution.insert(0,node['state'])

      return (solution_cost, solution)

    # Getting all actions the state can do
    actions = direcConnected[node['state']]
    for son_state in actions:
      son_node = {'state': son_state, 'path_cost' : node['path_cost'] + stepCost(node['state'], son_state), 'came_from' : node['state']}

      # Search if son is in explored
      in_explored = False
      if son_state in explored:
        in_explored = True

      # Search if son is in frontier
      in_frontier = False
      for frontier_node in frontier:
        if frontier_node[1]['state'] == son_state:
          in_frontier = True
          break
      
      # If son isn't already explored nand on frontier we add it to the frontier
      if not in_frontier and not in_explored:
       frontier.append((node_depth + 1, son_node))

  return None

# Searchs nodes by maximum limit within a increasingly depth_limit
# Returns (None) if there is no answer
# Returns (path cost, [initial state, ...,  objective]) if finds the solution
def iterativeDeepeningSearch(state):
  depth_limit = 0
  while True:
    result = limitedDeepeningSearch(state, depth_limit)
    if result is None:
      return None
    elif result[1] is None:
      depth_limit += 1
    else:
      return result

# Definindo as buscas heurísticas

**Busca A***

In [ ]:
import heapq
import time

# Searchs nodes by minimum  path cost + h = f
# Returns None if there is no solution
# Returns (path cost, [initial state, ...,  objective]) if finds the solution
def AstarSearch(state):
    global nodesAstar # to count the amount of nodes generated
    global direcConnected # to access all the possible actions of a state

    node = {'state' : state, 'path_cost' : 0, 'came_from' : state}
    frontier = [] # we use a heap queue ordered by the path cost + h(state)
    explored = {} # dictionary with {name_of_state : node} pairs
    
    # time.perf_counter() is in the tuple to solve drawed comparisons between paths of equal costs
    heapq.heappush(frontier, (node['path_cost'] + h(node['state']), time.perf_counter(), node))
    nodesAstar += 1

    while True:
      # If there are no more nodes to explore and we didn't find the solution
      if (len(frontier) == 0): 
        return None
      
      # Picking the node with smaller path cost
      node = heapq.heappop(frontier)[2] 
      explored.update({node['state'] : node})
      nodesAstar += 1

      # If we achieved the objective, return the path and total path cost
      if (objectiveTest(node['state'])):
        solution = []
        solution_cost = node['path_cost']

        while node['path_cost'] != 0: # path_cost = 0 only for stater node
          solution.insert(0,node['state'])
          node = explored[node['came_from']]
        solution.insert(0,node['state'])

        return (solution_cost, solution)

      # Getting all actions the state can do
      actions = direcConnected[node['state']]
      for son_state in actions:
        son_node = {'state': son_state, 'path_cost' : node['path_cost'] + stepCost(node['state'], son_state), 'came_from' : node['state']}

        # Search if son is in explored
        in_explored = False
        if son_state in explored:
          in_explored = True

        # Search if son is in frontier
        in_frontier = False
        for frontier_node in frontier:
          if frontier_node[2]['state'] == son_state:
            in_frontier = True
            break
        
        # If son isn't already explored nand on frontier we add it to the frontier
        if not in_frontier and not in_explored:
          heapq.heappush(frontier, (son_node['path_cost'] + h(son_node['state']), time.perf_counter(), son_node))

        # If son is already explored, but this new path is smaller
        elif in_frontier and frontier_node[0] > son_node['path_cost']:
          frontier_node = (son_node['path_cost'] + h(son_node['state']), time.perf_counter(), son_node)
          heapq.heapify(frontier)


**Busca IDA***

In [ ]:
import heapq
import time

# Searchs only nodes with path cost + h = f <= f_limit by minimum f
# Returns None if there is no solution
# Returns (f(node), None) if node has the smaller f in the frontier and still surpasses the f_limit
# Returns (path cost, [initial state, ...,  objective]) if finds the solution
def limitedAstarSearch(state, limit):
    global nodesIDAstar # to count the amount of nodes generated
    global direcConnected # to access all the possible actions of a state

    node = {'state' : state, 'path_cost' : 0, 'came_from' : state}
    frontier = [] # we use a heap queue ordered by the path cost + h(state)
    explored = {} # dictionary with {name_of_state : node} pairs
    heapq.heappush(frontier, (node['path_cost'] + h(node['state']), time.perf_counter(), node))
    nodesIDAstar += 1

    while True:
      # If there are no more nodes to explore and we didn't find the solution
      if (len(frontier) == 0): 
        return None
      
      # Picking the node with smaller path cost
      node = heapq.heappop(frontier)[2] 
      explored.update({node['state'] : node})
      nodesIDAstar += 1

      # If path cost + h of the node surpasses the limit, we stop the search
      if node['path_cost'] + h(node['state']) > limit:
        return (node['path_cost'] + h(node['state']), None)

      # If we achieved the objective, return the path and total path cost
      if (objectiveTest(node['state'])):
        solution = []
        solution_cost = node['path_cost']

        while node['path_cost'] != 0: # path_cost = 0 only for stater node
          solution.insert(0,node['state'])
          node = explored[node['came_from']]
        solution.insert(0,node['state'])

        return (solution_cost, solution)

      # Getting all actions the state can do
      actions = direcConnected[node['state']]
      for son_state in actions:
        son_node = {'state': son_state, 'path_cost' : node['path_cost'] + stepCost(node['state'], son_state), 'came_from' : node['state']}

        # Search if son is in explored
        in_explored = False
        if son_state in explored:
          in_explored = True

        # Search if son is in frontier
        in_frontier = False
        for frontier_node in frontier:
          if frontier_node[2]['state'] == son_state:
            in_frontier = True
            break
        
        # If son isn't already explored nand on frontier we add it to the frontier
        if not in_frontier and not in_explored:
          heapq.heappush(frontier, (son_node['path_cost'] + h(son_node['state']), time.perf_counter(), son_node))

        # If son is already explored, but this new path is smaller
        elif in_frontier and frontier_node[0] > son_node['path_cost']:
          frontier_node = (son_node['path_cost'] + h(son_node['state']), time.perf_counter(), son_node)
          heapq.heapify(frontier)

# Searchs only nodes with path cost + h = f <= f_limit by minimum f with a increasingly limit
# Returns None if there is no solution
# Returns (path cost, [initial state, ...,  objective]) if finds the solution
def IDAstarSearch (state):
  f_limit = 0
  while True:
    result = limitedAstarSearch(state, f_limit)
    if result is None:
      return None
    elif result[1] is None:
      f_limit = result[0]
    else:
      return result

Realizando um pequeno teste para a busca de um caminho de Porto Alegre a Maceió

In [ ]:
objective = 'Maceió'

nodesIterativeDeepening = 0
nodesUniformCost = 0
nodesAstar = 0
nodesIDAstar = 0

print (iterativeDeepeningSearch('Porto Alegre'), nodesIterativeDeepening, 'nós')
print (uniformCostSearch('Porto Alegre'), nodesUniformCost, 'nós')
print (AstarSearch('Porto Alegre'), nodesAstar, 'nós')
print (IDAstarSearch('Porto Alegre'), nodesIDAstar, 'nós')

(3921, ['Porto Alegre', 'Curitiba', 'São Paulo', 'Rio de Janeiro', 'Vitória', 'Salvador', 'Aracaju', 'Maceió']) 131 nós
(3921, ['Porto Alegre', 'Curitiba', 'São Paulo', 'Rio de Janeiro', 'Vitória', 'Salvador', 'Aracaju', 'Maceió']) 17 nós
(3921, ['Porto Alegre', 'Curitiba', 'São Paulo', 'Rio de Janeiro', 'Vitória', 'Salvador', 'Aracaju', 'Maceió']) 14 nós
(3921, ['Porto Alegre', 'Curitiba', 'São Paulo', 'Rio de Janeiro', 'Vitória', 'Salvador', 'Aracaju', 'Maceió']) 118 nós


# Obtendo resultados

Testaremos as quatro buscas com todas as 27*27 = 729 combinações de estados iniciais com estados objetivos possíveis. Para cada combinação, armazenaremos o tempo decorrido e a quantidade de nós gerados nas quatro buscas.

In [ ]:
timeScores = list()
nodeScores = list()
listOfStates = list(direcConnected.keys())
for i in range(0, len(listOfStates)):
  timeScores.append(list())
  nodeScores.append(list())
  initialState = listOfStates[i]
  for j in range(0, len(listOfStates)):
    objective = listOfStates[j]
    nodesUniformCost = nodesIterativeDeepening = nodesAstar = nodesIDAstar = 0
   
    timeUniformCost = time.perf_counter()
    uniformCostSearch(initialState)
    timeUniformCost = time.perf_counter() - timeUniformCost

    timeIterativeDeepening = time.perf_counter()
    iterativeDeepeningSearch(initialState)
    timeIterativeDeepening = time.perf_counter() - timeIterativeDeepening

    timeAstar = time.perf_counter()
    AstarSearch(initialState)
    timeAstar = time.perf_counter() - timeAstar

    timeIDAstar = time.perf_counter()
    IDAstarSearch(initialState)
    timeIDAstar = time.perf_counter() - timeIDAstar

    timeScores[i].append([timeUniformCost, timeIterativeDeepening, timeAstar, timeIDAstar])
    nodeScores[i].append([nodesUniformCost, nodesIterativeDeepening, nodesAstar, nodesIDAstar])

# Analisando os resultados

In [87]:
legenda = {0 : 'Custo Uniforme',
           1 : 'Aprofundamento Iterativo',
           2 : 'A*',
           3 : 'IDA*'}


print ('Algoritmos mais lentos em cada busca')
s_scores = [0]*4
for i in range(0, 27):
  for j in range(0, 27):
    result = timeScores[i][j].index(max(timeScores[i][j]))
    s_scores[result] += 1
for i in range (0, 4):
  print (legenda[i], '=', s_scores[i])


print('\nAlgoritmos que geraram mais nós em cada busca')
mn_scores = [0]*4
for i in range(0, 27):
  for j in range(0, 27):
    result = nodeScores[i][j].index(max(nodeScores[i][j]))
    mn_scores[result] += 1
for i in range (0, 4):
  print (legenda[i], '=', mn_scores[i])

print ('\nAlgoritmos mais rápidos em cada busca')
f_scores = [0]*4
for i in range(0, 27):
  for j in range(0, 27):
    result = timeScores[i][j].index(min(timeScores[i][j]))
    f_scores[result] += 1
for i in range (0, 4):
  print (legenda[i], '=', f_scores[i])

print('\nAlgoritmos que geraram menos nós em cada busca')
ln_scores = [0]*4
for i in range(0, 27):
  for j in range(0, 27):
    result = nodeScores[i][j].index(min(nodeScores[i][j]))
    ln_scores[result] += 1
for i in range (0, 4):
  print (legenda[i], '=', ln_scores[i])

Algoritmos mais lentos em cada busca
Custo Uniforme = 11
Aprofundamento Iterativo = 91
A* = 0
IDA* = 627

Algoritmos que geraram mais nós em cada busca
Custo Uniforme = 30
Aprofundamento Iterativo = 389
A* = 0
IDA* = 310

Algoritmos mais rápidos em cada busca
Custo Uniforme = 283
Aprofundamento Iterativo = 46
A* = 400
IDA* = 0

Algoritmos que geraram menos nós em cada busca
Custo Uniforme = 50
Aprofundamento Iterativo = 30
A* = 649
IDA* = 0


In [88]:
import plotly.io as pio

labels = ["Custo Uniforme", "Aprofundamento Iterativo","A*", "IDA*"]
fig = dict({
    "data": [{"type": "bar",
              "y": s_scores,
              "x": labels}],
    "layout": {"title": {"text": "Algoritmos mais lentos em cada busca"}}
})

pio.show(fig)

In [89]:
import plotly.io as pio

labels = ["Custo Uniforme", "Aprofundamento Iterativo","A*", "IDA*"]
fig = dict({
    "data": [{"type": "bar",
              "y": mn_scores,
              "x": labels}],
    "layout": {"title": {"text": "Algoritmos que geraram mais nós em cada busca"}}
})

pio.show(fig)

In [90]:
import plotly.io as pio

labels = ["Custo Uniforme", "Aprofundamento Iterativo","A*", "IDA*"]
fig = dict({
    "data": [{"type": "bar",
              "y": f_scores,
              "x": labels}],
    "layout": {"title": {"text": "Algoritmos mais rápidos em cada busca"}}
})

pio.show(fig)

In [91]:
import plotly.io as pio

labels = ["Custo Uniforme", "Aprofundamento Iterativo","A*", "IDA*"]
fig = dict({
    "data": [{"type": "bar",
              "y": ln_scores,
              "x": labels,
              }],
    "layout": {"title": {"text": "Algoritmos que geraram menos nós em cada busca"}}
})

pio.show(fig)

In [92]:

timeCost = []
timeDeep = []
timeA = []
timeIDA = []

nodeCost = []
nodeDeep = []
nodeA = []
nodeIDA = []

for i in range(0, 27):
  for j in range(0, 27):
    timeCost.append(timeScores[i][j][0])
    timeDeep.append(timeScores[i][j][1])
    timeA.append(timeScores[i][j][2])
    timeIDA.append(timeScores[i][j][3])
    nodeCost.append(nodeScores[i][j][0])
    nodeDeep.append(nodeScores[i][j][1])
    nodeA.append(nodeScores[i][j][2])
    nodeIDA.append(nodeScores[i][j][3])

avgTimeCost = 0
avgTimeDeep = 0
avgTimeA = 0
avgTimeIDA = 0
avgNodeCost = 0
avgNodeDeep = 0
avgNodeA = 0
avgNodeIDA = 0
for i in range(0, len(timeCost)):
  avgTimeCost += timeCost[i]
  avgTimeDeep += timeDeep[i]
  avgTimeA += timeA[i]
  avgTimeIDA += timeIDA[i]
  avgNodeCost += nodeCost[i]
  avgNodeDeep += nodeDeep[i]
  avgNodeA += nodeA[i]
  avgNodeIDA += nodeIDA[i]

avgTimeCost /= len(timeCost) *1000
avgTimeDeep /= len(timeCost) *1000
avgTimeA /= len(timeCost) *1000
avgTimeIDA /= len(timeCost) *1000
avgNodeCost /= len(timeCost) *1000
avgNodeDeep /= len(timeCost) *1000
avgNodeA /= len(timeCost) *1000
avgNodeIDA /= len(timeCost) *1000

In [93]:
import plotly.io as pio

tmp = [avgTimeCost, avgTimeDeep, avgTimeA, avgTimeIDA]
labels = ["Custo Uniforme", "Aprofundamento Iterativo","A*", "IDA*"]
fig = dict({
    "data": [{"type": "bar",
              "y": tmp,
              "x": labels,
              }],
    "layout": {"title": {"text": "Média de tempo (em ms) por algoritmo"}}
})

pio.show(fig)

In [94]:
import plotly.io as pio

tmp = [avgNodeCost, avgNodeDeep, avgNodeA, avgNodeIDA]
labels = ["Custo Uniforme", "Aprofundamento Iterativo","A*", "IDA*"]
fig = dict({
    "data": [{"type": "bar",
              "y": tmp,
              "x": labels,
              }],
    "layout": {"title": {"text": "Média de nós gerados por algoritmo"}}
})

pio.show(fig)